In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Embedding
from tensorflow.keras.layers import SpatialDropout1D, Conv1D, Bidirectional, LSTM, Dense, Input, Dropout, GlobalMaxPooling1D
from keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy



In [3]:
df = pd.read_csv(r'/content/gdrive/MyDrive/Final_Dataset.tsv',delimiter='\t')

In [4]:
print("On a  {} doublons dans Data.".format(df.duplicated().sum()))

On a  3030 doublons dans Data.


In [5]:
df.drop_duplicates(inplace = True)

In [ ]:
print(df.head())

In [ ]:
positive_reviews = df[df['Label'] > 3]
positive_reviews

,Label,Text
0,4,من أمتع ما قرأت من روايات بلا شك. وحول الشك تد...
1,4,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...
3,5,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...
4,5,"""عزازيل"" هو اسم رواية يوسف زيدان الثانية و ال..."
6,4,هذا هو الأدب . هذه هي الرواية !
...,...,...
63245,4,رواية جميلة . اثار انتباهي فيها سرعة اتخاذ ها...
63249,5,فلسفه بودن يا نبودنمان هم هنوز نيست.عجبا!
63250,4,حينما يدعي هملت الجنون يكون أكثر حكمة
63251,5,اجمل مسرحية ألفت في تاريخ الادب الانجليزي


In [ ]:
negative_reviews = df[df['Label'] < 3]
negative_reviews


,Label,Text
2,1,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...
35,2,انا كان عندى شغف قوى جدا لاقرأه وبعد ان تعمقت...
84,1,منذ مدة طويلة وأنا لا أقرأ الروايات ولكني اختر...
87,2,هو ممتع فى المجمل احداثه مشوقه هيبا شخصيته غر...
89,2,ذكر الكاتب في مقدمته أنها ترجمة لمخطوطات كتبه...
...,...,...
63225,2,الرواية شيقة واستمتعت في قراءتها وأعجبت بشخصي...
63226,1,عن خادمة تسافر من الفلبين إلى الكويت لا شيء ج...
63231,2,""" جميلة بالرغم من بساطة اسلوب الكاتب """
63233,2,تعليق على صفحة 179: نقله غير متوقعه وغير جميله...


In [ ]:
data = pd.concat([positive_reviews, negative_reviews], ignore_index = True)
data.head()

,Label,Text
0,4,من أمتع ما قرأت من روايات بلا شك. وحول الشك تد...
1,4,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...
2,5,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...
3,5,"""عزازيل"" هو اسم رواية يوسف زيدان الثانية و ال..."
4,4,هذا هو الأدب . هذه هي الرواية !


In [ ]:
df_train,df_test=train_test_split(data,test_size=0.3)

In [ ]:
MAX_VOCAB_SIZE=250000
tokenizer=Tokenizer(num_words=MAX_VOCAB_SIZE,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(df_train['Text'])
sequences_train = tokenizer.texts_to_sequences(df_train['Text'])
sequences_test = tokenizer.texts_to_sequences(df_test['Text'])

In [ ]:
word2idx=tokenizer.word_index
V=len(word2idx)
V

215241

In [ ]:
MAX_SEQUENCE_LENGTH=3456
data_train=pad_sequences(sequences_train,maxlen = MAX_SEQUENCE_LENGTH)


In [ ]:
data_test=pad_sequences(sequences_test,maxlen = MAX_SEQUENCE_LENGTH)

In [ ]:
import gensim
Embedding_dim=300
file='/content/gdrive/MyDrive/tweet_cbow_300/tweets_cbow_300'
!iconv -f ISO-8859-1 -t UTF-8 /content/gdrive/MyDrive/tweet_cbow_300/tweets_cbow_300> /content/gdrive/MyDrive/tweets_utf8.txt
file_utf8 = '/content/gdrive/MyDrive/tweets_utf8.txt'
# Initialize an empty embeddings index dictionary
EMBEDDINGS_MATRIX = np.zeros ((V+1, Embedding_dim)) #EMBEDDINGS MATRIX-11
model = gensim.models.Word2Vec.load(file)
for word, i in word2idx.items():
  embedding_vector = None
  if word != "<OOV>":
    if word in model.wv:
      embedding_vector = model.wv[word]
EMBEDDINGS_MATRIX[i] = embedding_vector


In [ ]:
EMBEDDINGS_MATRIX = np.nan_to_num(EMBEDDINGS_MATRIX)

In [ ]:
D = 300

# Note: we actually want to the size of the embedding to (V + 1) x D,
# because the first index starts from 1 and not 0.
# Thus, if the final index of the embedding matrix is V,
# then it actually must have size V + 1.

i = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = Embedding(V + 1, D,weights = [EMBEDDINGS_MATRIX])(i)
x = Conv1D(64, 2, activation='relu')(x)
# x = MaxPooling1D(3)(x)
# x = Conv1D(64, 3, activation='relu')(x)
# x = MaxPooling1D(3)(x)
# x = Conv1D(128, 3, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(6)(x)

model = Model(i, x)

In [ ]:
model.compile(
  loss='binary_crossentropy',
  optimizer=Adam(learning_rate=0.001),
  metrics=['accuracy']
)


print('Training model...')
r = model.fit(
  data_train,
  df_train['Label'],
  batch_size = 60,
  epochs=5,

  validation_data=(data_test, df_test['Label'])
)

Training model...
Epoch 1/5
567/567 [==============================] - 2028s 4s/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/5
567/567 [==============================] - 2011s 4s/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/5
567/567 [==============================] - 2037s 4s/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/5
567/567 [==============================] - 2086s 4s/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/5
567/567 [==============================] - 2220s 4s/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00


In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend();

NameError: ignored